In [2]:
import pymysql
import ujson
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import torch
np.set_printoptions(threshold = np.inf, suppress = True)

In [3]:
db = pymysql.connect(host="10.10.10.10", port=3306, user="readonly", password="readonly", db="acekg", charset="utf8")

In [4]:
cur = db.cursor()
sql = 'select distinct question from dde_paper_abstract_QA_ner'
cur.execute(sql)
result = cur.fetchall()

In [9]:
relation = []
for one in result:
    relation.append(one[0])
print(relation)

['aim', 'problem', 'method', 'methor_base', 'one_sentence_summary', 'the_deferences_comparing_to_previous_methods', 'experiment', 'results', 'result', 'the_method_is_based_on', 'result_outperform', 'performance_improvements', 'model', 'method_dif', 'dataset', 'model_dif', 'model_base', 'the_deferences_comparing_to_previous_models', 'the_model_is_based_on', 'the_deferences_comparing_to_previous_frameworks', 'framework', 'framework_base', 'algorithm_base', 'algorithm', 'the_algorithm_is_based_on']


In [10]:
rel = {}
rel['aim'] = ['aim', 'problem','one_sentence_summary','dataset']
rel['method'] = ['method', 'methor_base','the_deferences_comparing_to_previous_methods','the_method_is_based_on','method_dif']
rel['result'] = ['experiment', 'results', 'result','result_outperform', 'performance_improvements']
rel['model'] = ['model','model_dif', 'model_base', 'the_deferences_comparing_to_previous_models', 'the_model_is_based_on']
rel['framework'] = ['the_deferences_comparing_to_previous_frameworks', 'framework', 'framework_base']
rel['algorithm'] = ['algorithm_base', 'algorithm', 'the_algorithm_is_based_on']

In [11]:
relation_map = {}
for one in relation:
    for key in rel.keys():
        if one in rel[key]:
            relation_map[one]=key

In [12]:
relation_map

{'aim': 'aim',
 'problem': 'aim',
 'method': 'method',
 'methor_base': 'method',
 'one_sentence_summary': 'aim',
 'the_deferences_comparing_to_previous_methods': 'method',
 'experiment': 'result',
 'results': 'result',
 'result': 'result',
 'the_method_is_based_on': 'method',
 'result_outperform': 'result',
 'performance_improvements': 'result',
 'model': 'model',
 'method_dif': 'method',
 'dataset': 'aim',
 'model_dif': 'model',
 'model_base': 'model',
 'the_deferences_comparing_to_previous_models': 'model',
 'the_model_is_based_on': 'model',
 'the_deferences_comparing_to_previous_frameworks': 'framework',
 'framework': 'framework',
 'framework_base': 'framework',
 'algorithm_base': 'algorithm',
 'algorithm': 'algorithm',
 'the_algorithm_is_based_on': 'algorithm'}

# qid idq

In [13]:
f_pqid = open('data/pqid.txt', 'r', encoding='utf8')
qid = {}
idq = {}
flag = 0
for line in f_pqid:
    c = line[:-1].replace("\"","").split('\t')
    if str(c[0]) + "@" + relation_map[str(c[1])] in qid.keys():
        continue
    qid[str(c[0]) + "@" + relation_map[str(c[1])]] = flag
    idq[flag] = str(c[0]) + "@" + relation_map[str(c[1])]
    flag += 1

In [15]:
idq

{0: '433977@method',
 1: '433977@aim',
 2: '527599@method',
 3: '527599@aim',
 4: '527599@result',
 5: '1036524@result',
 6: '1036524@method',
 7: '1036524@aim',
 8: '1227799@aim',
 9: '1227799@method',
 10: '2488495@result',
 11: '2488495@method',
 12: '2488495@aim',
 13: '3032562@aim',
 14: '3032562@result',
 15: '3032562@method',
 16: '3297940@aim',
 17: '3297940@result',
 18: '3558404@aim',
 19: '3558404@result',
 20: '3558404@method',
 21: '3563299@result',
 22: '3563299@method',
 23: '3563299@aim',
 24: '3616883@aim',
 25: '3616883@result',
 26: '3616883@method',
 27: '3616883@model',
 28: '3823721@result',
 29: '3823721@method',
 30: '3823721@aim',
 31: '4147986@aim',
 32: '4147986@method',
 33: '4147986@result',
 34: '4273597@aim',
 35: '4273597@result',
 36: '4768141@aim',
 37: '4768141@result',
 38: '4768141@method',
 39: '5099496@aim',
 40: '5099496@result',
 41: '5130908@result',
 42: '5130908@aim',
 43: '5401876@aim',
 44: '5401876@method',
 45: '5929723@aim',
 46: '592972

# benchamrk

In [16]:
f_benchmark_paper_id = open('data/benchmark_paper.txt', 'r', encoding='utf8')
f_benchmark_res_top3 = open('data/benchmark_result.txt', 'r', encoding='utf8')

bm_paper = []
for line in f_benchmark_paper_id:
    bm_paper.append(line[:-1])

bm_answer = []
for line in f_benchmark_res_top3:
    bm_answer.append(line[:-1])

In [17]:
bm_paper[:2], bm_answer[:2]

(['420342715', '87250949'], ['21042067', '21042097'])

In [18]:
def get_benchmark(paper_id):
    cur = db.cursor()
    sql='SELECT paper_id, question, abs_score, title_score, qa_score, word_len, letter_len, complex_len, ae_id FROM dde_paper_abstract_QA_ner WHERE paper_id = ' + str(paper_id)
    cur.execute(sql)
    one_paper_bm = []
    try:
        result = cur.fetchall()
        for res in result:
            para = []
            for i in range(2, 8):
                para.append(round(float(res[i]),5))
            one_line_bm = "qid:"+str(qid[str(res[0])+"@"+relation_map[str(res[1])]])+" aeid:"+str(res[8])+" 1:"+str(round(para[0], 5))+" 2:"+str(round(para[1], 5))+" 3:"+str(round(para[2], 5))+" 4:"+str(round(para[3], 5))+" 5:"+str(round(para[4], 5))+" 6:"+str(round(para[5], 5))
            one_paper_bm.append((one_line_bm, res[8]))
        return one_paper_bm
    except Exception as e:
        print(e)
        return one_paper_bm

In [19]:
get_benchmark(420342715)[:10]

[('qid:3126 aeid:17739994 1:0.16046 2:0.22299 3:0.36968 4:2.0 5:17.0 6:1.0',
  17739994),
 ('qid:3126 aeid:17740016 1:0.13081 2:0.14377 3:0.32 4:2.0 5:17.0 6:1.0',
  17740016),
 ('qid:3126 aeid:17740034 1:0.09939 2:0.26934 3:0.18308 4:1.0 5:10.0 6:0.0',
  17740034),
 ('qid:3126 aeid:17740052 1:0.12657 2:0.10073 3:0.3283 4:1.0 5:7.0 6:0.0',
  17740052),
 ('qid:3126 aeid:17740067 1:0.12446 2:0.25913 3:0.17614 4:1.0 5:9.0 6:0.0',
  17740067),
 ('qid:3126 aeid:17740093 1:0.10691 2:0.2087 3:0.14186 4:1.0 5:9.0 6:0.0',
  17740093),
 ('qid:3126 aeid:17740110 1:0.11803 2:0.15885 3:0.10798 4:1.0 5:11.0 6:0.0',
  17740110),
 ('qid:3126 aeid:17740126 1:0.11565 2:0.03756 3:0.2475 4:1.0 5:6.0 6:0.0',
  17740126),
 ('qid:3126 aeid:17740142 1:0.06985 2:0.02143 3:0.15978 4:1.0 5:10.0 6:0.0',
  17740142),
 ('qid:3126 aeid:17740156 1:0.0476 2:0.15894 3:0.10804 4:1.0 5:6.0 6:0.0',
  17740156)]

In [20]:
benchmark_tmp = []
for pid in tqdm(bm_paper):
    benchmark_tmp += get_benchmark(paper_id=pid)

In [23]:
benchmark = []
for tmp in benchmark_tmp:
    if str(tmp[1]) in bm_answer:
        bm_tmp = '1 ' + tmp[0]
    else:
        bm_tmp = '0 ' + tmp[0]
    benchmark.append(bm_tmp)

In [24]:
benchmark[:10]

['1 qid:3126 aeid:17739994 1:0.16046 2:0.22299 3:0.36968 4:2.0 5:17.0 6:1.0',
 '1 qid:3126 aeid:17740016 1:0.13081 2:0.14377 3:0.32 4:2.0 5:17.0 6:1.0',
 '0 qid:3126 aeid:17740034 1:0.09939 2:0.26934 3:0.18308 4:1.0 5:10.0 6:0.0',
 '0 qid:3126 aeid:17740052 1:0.12657 2:0.10073 3:0.3283 4:1.0 5:7.0 6:0.0',
 '0 qid:3126 aeid:17740067 1:0.12446 2:0.25913 3:0.17614 4:1.0 5:9.0 6:0.0',
 '0 qid:3126 aeid:17740093 1:0.10691 2:0.2087 3:0.14186 4:1.0 5:9.0 6:0.0',
 '0 qid:3126 aeid:17740110 1:0.11803 2:0.15885 3:0.10798 4:1.0 5:11.0 6:0.0',
 '0 qid:3126 aeid:17740126 1:0.11565 2:0.03756 3:0.2475 4:1.0 5:6.0 6:0.0',
 '0 qid:3126 aeid:17740142 1:0.06985 2:0.02143 3:0.15978 4:1.0 5:10.0 6:0.0',
 '0 qid:3126 aeid:17740156 1:0.0476 2:0.15894 3:0.10804 4:1.0 5:6.0 6:0.0']

In [25]:
bm = open('data/benchmark.txt', 'w', encoding='utf8')
for line in benchmark:
    print(line[:-1], file=bm)

In [26]:
bm.close()
f_benchmark_paper_id.close()
f_benchmark_res_top3.close()
f_pqid.close()

# dataset

In [27]:
def get_dataset():
    dataset = []
    dp_list = []
    cur = db.cursor()
    sql='SELECT paper_id, question, abs_score, title_score,qa_score, word_len, letter_len, complex_len, mark, ae_id FROM dde_paper_abstract_QA_ner where mark is not null'
    cur.execute(sql)
    result = cur.fetchall()
    for res in result:
        para = []
        for i in range(2, 8):
            para.append(round(float(res[i]),5))
        one_line = str(res[8])+" qid:"+str(qid[str(res[0])+"@"+relation_map[str(res[1])]])+" aeid:"+str(res[9])+" 1:"+str(round(para[0], 5))+" 2:"+str(round(para[1], 5))+" 3:"+str(round(para[2], 5))+" 4:"+str(round(para[3], 5))+" 5:"+str(round(para[4], 5))+" 6:"+str(round(para[5], 5))
        dataset.append(one_line)
    return dataset

In [28]:
dataset = get_dataset()
len(dataset)

8247

In [29]:
dataset[1]

'0 qid:2464 aeid:14753826 1:0.02323 2:0.02874 3:0.04246 4:1.0 5:10.0 6:0.0'

# train test divide

In [34]:
def build_dateset(dataset):
    training_set = []
    testing_set = []
    dp_train = []
    dp_test = []
    for index in range(0, len(dataset)):
        c = dataset[index].split(" ")
        record = [int(c[0]), int(c[1].split(":")[1]), int(c[2].split(":")[1]), float(c[3].split(":")[1]), float(c[4].split(":")[1]), float(c[5].split(":")[1]), float(c[6].split(":")[1]), float(c[7].split(":")[1]),float(c[8].split(":")[1])]    
        if index < 4/5*len(dataset):
            training_set.append(record)
        else:
            testing_set.append(record)
    return training_set, testing_set

def build_benchmark(benchmark):
    benchmark_set = []
    for index in range(0, len(benchmark)):
        c = benchmark[index].split(" ")
        record = [int(c[0]), int(c[1].split(":")[1]), int(c[2].split(":")[1]), float(c[3].split(":")[1]), float(c[4].split(":")[1]), float(c[5].split(":")[1]), float(c[6].split(":")[1]), float(c[7].split(":")[1]),float(c[8].split(":")[1])]    
        benchmark_set.append(record)
    return benchmark_set

In [35]:
training_set, testing_set = build_dateset(dataset)
benchmark_set = build_benchmark(benchmark)


In [36]:
len(benchmark_set)

2619

In [37]:
training_set[:3]
trs = np.array(training_set)
tes = np.array(testing_set)
trs[:3], tes[:3]

(array([[       1.     ,     2464.     , 14753805.     ,        0.07169,
                0.11169,        0.16501,        1.     ,        8.     ,
                0.     ],
        [       0.     ,     2464.     , 14753826.     ,        0.02323,
                0.02874,        0.04246,        1.     ,       10.     ,
                0.     ],
        [       0.     ,     2464.     , 14753946.     ,        0.03952,
                0.0438 ,        0.06472,        1.     ,        9.     ,
                0.     ]]),
 array([[       0.     ,      260.     , 13025941.     ,        0.03194,
                0.0435 ,        0.05837,        1.     ,        8.     ,
                0.     ],
        [       0.     ,      260.     , 13025952.     ,        0.12283,
                0.008  ,        0.23426,        2.     ,       19.     ,
                1.     ],
        [       0.     ,      260.     , 13025990.     ,        0.17215,
                0.     ,        0.1505 ,        1.     ,       13

In [38]:
bms = np.array(benchmark_set)
bms[:3]

array([[       1.     ,     3126.     , 17739994.     ,        0.16046,
               0.22299,        0.36968,        2.     ,       17.     ,
               1.     ],
       [       1.     ,     3126.     , 17740016.     ,        0.13081,
               0.14377,        0.32   ,        2.     ,       17.     ,
               1.     ],
       [       0.     ,     3126.     , 17740034.     ,        0.09939,
               0.26934,        0.18308,        1.     ,       10.     ,
               0.     ]])

In [39]:
np.save('train.npy', trs)
np.save('test.npy', tes)
np.save('benchmark.npy', bms)